<a href="https://colab.research.google.com/github/ugursirvermez/PyTorch_Education/blob/main/05_pytorch_cv_cnn_model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CNN Yüksek Seviye Model Tasarlama: Model_2

##Modelin Aşamalarına Test
Modelin aşamalarını test etmek için yazılan kodlar, pasifler.

In [ ]:
#---------------------------------------------------------------TEST ETMEK İÇİN----------------------------------------------------------
'''
images = torch.randn(size=(32,3,64,64)) #resimleri batch=32, color_channel=3, size= 64x64
test_image = images[0] #Örnek resim

#nn.Conv2d() -> Bunun üzerine gideceğiz.
conv_layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=1, padding=0)
#Convolutional katmana doğru veriyi geçirme
conv_output = conv_layer(test_image.unsqueeze(0)) #hata olmasın diye 3 boyutlu veriyi unsqueeze ile 4 boyutlu yaptık.

#nn.Maxpool2d -> Conventional sonrası kullanılan katman
max_pool_layer = nn.MaxPool2d(kernel_size=2)
#conv_layer'dan veriyi geçirelim.
test_image_through_conv = conv_layer(test_image.unsqueeze(dim=0))
#max pool layer'a veriyi geçirelim.
test_image_through_conv_and_max_pool = max_pool_layer(test_image_through_conv)
'''
print("Önceki test kodları")
#---------------------------------------------------------------TEST BURADA BITIYOR------------------------------------------------------

Önceki test kodları


#MODEL_2
İçinde CNN, Pooling ve Sınıflandırma var.

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch import nn
from timeit import default_timer as timer # Zaman ayarları için var.
from tqdm.auto import tqdm #Eğitimi işlem barında göstermek için kullanacağız. Progess bar kütüphanesi
import requests
from pathlib import Path
if Path("helper_functions.py").is_file():
	print("helper_functions.py varsa indirme")
else:
	print("helper_function.py indir")
	request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
	with open("helper_functions.py", "wb") as f: #aynı dosyayı aç ve içeriğini yaz.
		f.write(request.content)

from helper_functions import plot_predictions, plot_decision_boundary, accuracy_fn #Daha önceden yazdığımız kodlardan alınma aslında.
#-------------------------------------------------------KÜTÜPHANELER--------------------------------------------------------------------
#Device tanımlama
device = "cuda" if torch.cuda.is_available() else "cpu"

#CNN Oluşturma
class FashionMNISTModelV2(nn.Module): #TinyVGG modeli yapacağız.
	def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
		super().__init__()
		#CNN için Conv2d kullanıyoruz. TinyVGG'de iki tane Conv modeli kullanıldığı için 2 kullandık.
		self.conv_block_1 = nn.Sequential(#kernel_size, stride ve padding bizim belirlediğimiz değerlerdir. Bunu anlamak için: https://poloclub.github.io/cnn-explainer/
												nn.Conv2d(in_channels = input_shape, out_channels = hidden_units, kernel_size=3, stride=1, padding=1),
												nn.ReLU(),
												nn.Conv2d(in_channels = hidden_units, out_channels = hidden_units, kernel_size=3, stride=1, padding=1),
												nn.ReLU(),
												nn.MaxPool2d(kernel_size=2) #Pooling Layer
												)
		#Şimdi 2.CNN katmanını kuruyoruz.
		self.conv_block_2 = nn.Sequential(
												nn.Conv2d(in_channels = hidden_units, out_channels = hidden_units, kernel_size=3, stride=1, padding=1),
												nn.ReLU(),
												nn.Conv2d(in_channels = hidden_units, out_channels = hidden_units, kernel_size=3, stride=1, padding=1),
												nn.ReLU(),
												nn.MaxPool2d(kernel_size=2)
												)
		#Şimdi doğrusal modele çıkarmak için yaptığımız self.Linear modeli uygulayacağız.
		self.classifier= nn.Sequential(nn.Flatten(),
																		 nn.Linear(in_features = hidden_units *7 *7, out_features = output_shape),
																		 nn.ReLU()) # doğrusal olmayan çıktı için ReLU gereklidir.

	def forward(self,x):
		x = self.conv_block_1(x)
		#print(f"Conv_1:{x.shape}") # ne oluyor bir bakarız.
		x = self.conv_block_2(x)
		#print(f"Conv_2:{x.shape}")
		x = self.classifier(x)
		return x # zaten output shape olacak.
#---------------------------------------------------------------MODEL BURADA BITIYOR------------------------------------------------------
#---------------------------------------MODEL_1 HAZIRLADIGIMIZ FONKSIYONLAR---------------------------------------------------------------
#TIMER KODU
#Zamana bağlı eğitimi yazdırmak için var.
def print_train_time(start: float, end: float, device: torch.device = None):
	total_time = end - start
	print(f"Eğitim Zamanı: {device}: {total_time} Saniye")
	return total_time

#TRAIN_STEP ve TEST_STEP FONKSİYONLARI
#TRAIN_STEP
def train_step(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module,
							 optimizer: torch.optim.Optimizer, accuracy_fn, device: torch.device = device):

	train_loss, train_acc = 0,0

	#Eğitime başlıyoruz.
	model.train()

	#Döngü içinde döngü yapalım.
	for batch, (X, y) in enumerate(data_loader): #X -> Image, y -> Label
		#Cihazları ayarlayalım.
		X, y = X.to(device), y.to(device)

		y_pred = model_2(X) #Forward edilecek.

		#Loss'u her batch için yapacağız.
		loss = loss_fn(y_pred, y)
		train_loss += loss # eğitim kaybını kümeleyelim.
		train_acc = accuracy_fn(y_true = y, y_pred = y_pred.argmax(dim=1))

		#Batch için Optimizer
		optimizer.zero_grad()
		#Loss Backward
		loss.backward()
		#Optimizer step
		optimizer.step()

	#Train_dataloader'daki kayba bakalım.
	train_loss /= len (data_loader)
	train_acc /= len(data_loader)
	print(f"Train Loss: {train_loss: .5f} | Train Acc: {train_acc: .2f}")

#TEST_STEP
def test_step(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module,
							optimizer: torch.optim.Optimizer, accuracy_fn, device: torch.device = device):

	test_loss, test_acc = 0, 0
	model.eval()
	# inference modu çalıştıralım.
	with torch.inference_mode():
		for X_test, y_test in data_loader:

			X_test, y_test = X_test.to(device), y_test.to(device)

			test_pred = model(X_test)

			test_loss += loss_fn(test_pred, y_test)
			test_acc += accuracy_fn(y_true = y_test, y_pred = test_pred.argmax(dim=1))

		test_loss /= len(data_loader)
		test_acc /= len(data_loader)
		print(f"Test Loss: {test_loss: .5f} | Test Acc: {test_acc:.2f}%\n")

#Model_2.eval() şeyleri
def eval_model(model: torch.nn.Module, data_loader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, accuracy_fn):
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # Make predictions with the model
            y_pred = model(X)

            # Accumulate the loss and accuracy values per batch
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y,
                                y_pred=y_pred.argmax(dim=1)) # For accuracy, need the prediction labels (logits -> pred_prob -> pred_labels)

        # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)
        acc /= len(data_loader)

    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}


#---------------------------------------MODEL_1 HAZIRLADIGIMIZ FONKSIYONLAR BITTI----------------------------------------------------------

#TRAIN_DATA
train_data = datasets.FashionMNIST(root="data", train=True, download=True, transform = torchvision.transforms.ToTensor(), target_transform = None)

#TEST_DATA
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform = torchvision.transforms.ToTensor(), target_transform = None)
#TRAIN DATA CLASS
class_names = train_data.classes

#DATALOADER
BATCH_SIZE = 32

#Veri setini yenilemeye çevirelim. Train_Data
train_dataloader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True)
#Test_Data
test_dataloader = DataLoader(dataset = test_data, batch_size = BATCH_SIZE, shuffle = False)
#Eğitilmiş verilerin içinde ne var?
train_features_batch , train_labels_batch = next(iter(train_dataloader))

#MODELİ ÇALIŞTIRALIM!
torch.manual_seed(42)
torch.cuda.manual_seed(42)
model_2 = FashionMNISTModelV2(input_shape = 1, hidden_units= 10, output_shape=len(class_names)).to(device) #Model tanımlandı.

#rastgele image olusturalım.
rand_image_tensor = torch.randn(size=(1, 28, 28))

#Model_2'ye sokalım.
#model_2(rand_image_tensor.unsqueeze(0).to(device))

#Loss ve Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(), lr=0.1)

#Training/Testing Fonksiyonları
#Timer start
Train_Start_timer = timer()

#dongu
epochs = 3
for epoch in tqdm(range(epochs)):
	print(f"Epoch:{epoch}\n----")
	#Train uygula
	train_step(model= model_2, data_loader = train_dataloader, loss_fn = loss_fn, optimizer=optimizer, accuracy_fn= accuracy_fn, device = device)
	#Test Uygula
	test_step(model= model_2, data_loader = test_dataloader, loss_fn = loss_fn, optimizer=optimizer, accuracy_fn= accuracy_fn, device = device)

#Timer End
Train_End_timer = timer()
#Timer Calc
total_train_time_model_2 = print_train_time(Train_Start_timer, Train_End_timer, device)

#Değerlendirme ve Sonuç yazdırma
model_2_results = eval_model(model=model_2, data_loader = test_dataloader, loss_fn = loss_fn, accuracy_fn = accuracy_fn)
print("İşlemler Bitti!")

helper_functions.py varsa indirme


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:0
----
Train Loss:  0.89260 | Train Acc:  0.04
Test Loss:  0.62134 | Test Acc: 81.85%

Epoch:1
----
Train Loss:  0.56656 | Train Acc:  0.04
Test Loss:  0.55760 | Test Acc: 82.82%

Epoch:2
----
Train Loss:  0.52327 | Train Acc:  0.05
Test Loss:  0.54754 | Test Acc: 82.85%

Eğitim Zamanı: cpu: 161.47594985500018 Saniye
İşlemler Bitti!


Yukarıdaki veri sonucu manual_seed'i tekrar yapıp yapmamaya göre değişiklik gösterecektir. Normalde bir daha seed yaptığımda % 80 en az başarı sağlıyor.